In [1]:
import os
import json

import pandas as pd

## Load the Dataset

In [2]:
MYCHART_DIR = "data/raw_datasets/mychart"
MYDOC_DIR = "data/raw_datasets/mydoc"
MYINFOGRAPHIC_DIR = "data/raw_datasets/myinfographic"

In [3]:
mychart = json.load(open(f"{MYCHART_DIR}/annot_wo_answer.json", "r"))
mydoc = json.load(open(f"{MYDOC_DIR}/annot_wo_answer.json", "r"))
myinfographic = json.load(open(f"{MYINFOGRAPHIC_DIR}/annot_wo_answer.json", "r"))

In [4]:
test_outputs = {
    "id": [],
    "pred": [],
    "split": [],
}

In [5]:
inference_results_sources = ["nous-hermes-docile", "llava-1-6-vicuna-13b-hf"]

In [6]:
for docs_list in [mychart, mydoc, myinfographic]:
    for doc in docs_list:
        test_outputs["id"].append(doc["id"])
        answer = "dummy"
        for inference_results_source in inference_results_sources:
            filename = f"inference_results/{inference_results_source}/{doc['id']}.txt"
            if os.path.exists(filename):
                with open(filename, "r") as f:
                    answer = f.read().strip().replace("\n", " ")
                break
        if answer.strip() == "":
            answer = "dummy"
        test_outputs["pred"].append(answer)
        test_outputs["split"].append("public")

In [7]:
train_val_datasets = [
    'docvqa',
    'funsd',
    'iconqa_choose_txt',
    'iconqa_fill_in_blank',
    'infographicvqa',
    'tabfact',
    'textbookqa',
    'wildreceipt',
    'wtq',
    'websrc'
]

In [8]:
llava_eval_models = [
    "phase-1--llava-v1.5-13b",
    "phase-1--llava-v1.5-7b-MMMU-epoch-10",
]
model_results_map = {
    model: {
        "_".join(key.split("_")[:-1]): value
        for key, value in results.items()
    }
    for model in llava_eval_models
    if (results := json.load(open(f"inference_results/{model}/output_datasets_all.json")))
}

In [9]:
split = "test"
for dataset_name in train_val_datasets:
    dataset_path = os.path.join("data/processed_data", dataset_name, f'converted_output_{split}.json')

    if os.path.exists(dataset_path):
        with open(dataset_path, 'r') as f:
            data = json.load(f)
    else:
        print(f'{dataset_path} do not exist')
        continue

    for doc in data:
        test_outputs["id"].append(f"{dataset_name}_{doc['id']}")
        # test_outputs["pred"].append(doc["conversations"][-1]["value"])
        answer = "dummy"
        for model in llava_eval_models:
            if (model_answer := model_results_map[model].get(f"{dataset_name}_{doc['id']}")) is not None:
                answer = model_answer
                break
        if answer == "dummy":
            for inference_results_source in inference_results_sources:
                filename = f"inference_results/{inference_results_source}/{dataset_name}_{doc['id']}.txt"
                if os.path.exists(filename):
                    with open(filename, "r") as f:
                        answer = f.read().strip().replace("\n", " ")
                    break
        if answer.strip() == "":
            answer = "dummy"
        test_outputs["pred"].append(answer)
        test_outputs["split"].append("public")

In [10]:
df = pd.DataFrame(test_outputs)

In [11]:
len(df)

3028

In [12]:
df.to_csv("test_output_8.csv", index=False, errors="strict")